# 9. Flask - Final Preparation and Mission Deployment

Our Mission Control is fully functional in the simulator (`localhost`), but now it's time for the final step: launching our application into the reality of the internet. In this final lesson, we will prepare our code for the production environment, improve its appearance, and perform the actual deployment, so that our application is accessible to anyone, from anywhere.

- Application `deployment` & `hosting` options
- Modifying the `configuration` for deployment
- Improving the appearance with `Bootstrap`

## 9.1. Pre-launch Check: Git & GitHub
For deployment (hosting), we will need two things: a **GitHub account**, where we will store the final plans for our application, and a **hosting service**, which will take these plans and build a functional, publicly accessible server from them.

* **GitHub:** Serves as our central and versioned repository for the source code. Most modern hosting services can connect directly to your GitHub repository and automatically deploy the application with every new change (known as CI/CD).

## 9.2. Rebuilding for a Production Machine
The development environment is designed for convenience, while production is for **security, performance, and stability**. We must therefore make several key adjustments.

### 9.2.1. Engine Swap: Production WSGI Server
The development server (`app.run()`) is not suitable for production. Instead, we will deploy **Gunicorn**, a robust and proven WSGI server.

1.  **Installing Gunicorn:**
    ```bash
    pip install gunicorn
    ```
2.  **Component List (`requirements.txt`):** The hosting service needs to know which libraries to install for our application. We will create a list:
    ```bash
    pip freeze > requirements.txt
    ```
3.  **Startup Sequence (`Procfile`):** We need to tell the hosting service how to start our application. We will create a file named `Procfile` (without an extension) in the root directory of the project with the following content:
    ```
    web: gunicorn wsgi:app
    ```
    *This command says: "For the web process (`web`), start the `gunicorn` server and tell it to find the `app` instance in the `wsgi.py` file."*

### 9.2.2. Securing Configuration: Environment Variables and `.env`
In production, we never leave sensitive data - like passwords and access credentials (e.g., `SECRET_KEY`) - written directly in the code. Therefore, we will use **environment variables** and an `.env` file to manage them. We will install `python-dotenv`.

1.  **Installing `python-dotenv`:**
    ```bash
    pip install python-dotenv
    ```
    
2.  **`.env` file:** In the root directory of the project, create a file named `.env`. **Never add this file to Git!** Add `.env` to your `.gitignore`.

    ```
    # FILE: .env
    # These are variables for the production environment

    SECRET_KEY=my_secret_password
    DATABASE_URL=sqlite:///production_database.db # Or another DB
    APP_ENV=production
    
    ```

3.  **Upgrading `config.py`:** we will modify our `Config` class:

    ```python
    # FILE: configuration.py
    import os # import OS
    from dotenv import load_dotenv # import load_dotenv

    load_dotenv()

    class Config:
        """Base configuration."""
        STATIC_FOLDER = 'static'
        TEMPLATES_FOLDER = 'templates'
        SECRET_KEY = os.environ.get('SECRET_KEY')

        SQLALCHEMY_TRACK_MODIFICATIONS = False
        SQLALCHEMY_DATABASE_URI = os.environ.get("DATABASE_URL")

    class ProdConfig(Config):
        APP_ENV = 'production'
        DEBUG = False

    class DevConfig(Config):
        APP_ENV = 'development'
        DEBUG = True
    ```

4.  **Upgrading the Application Factory:** in the factory, we will modify the configuration selection.
    ```python
    # FILE: app/__init__.py
    import os
    from flask import Flask
    from dotenv import load_dotenv
    from .config.configuration import DevConfig, ProdConfig

    load_dotenv() # loads .env values into environs

    def create_app():
        app = Flask(__name__)

        # CONFIGURATION - using ENV
        env = os.environ.get("APP_ENV", "production") # gets "APP_ENV" from environs, if not, default set to "production"
        if env == "development":
            app.config.from_object(DevConfig)
        else:
            app.config.from_object(ProdConfig)

        # ... db initialization, blueprint registration ...
        
        return app
    ```

## 9.3. Improving Appearance: Bootstrap Integration
To make our information panel look professional, we will use the CSS framework **Bootstrap** and can leverage the capabilities of an **extension for Flask**.

1. **Installing the extension:**
Documentation: https://bootstrap-flask.readthedocs.io/en/stable/basic/

```bash
    pip install bootstrap-flask
```

2. **Initialization in the application factory (app/`__init__.py`):**

In [ ]:
# __init__.py

import os
from flask import Flask
from dotenv import load_dotenv
from .config.configuration import DevConfig, ProdConfig
from flask_bootstrap import Bootstrap5 # import Bootstrap


load_dotenv() # loads values from .env

def create_app():

    app = Flask(__name__)

    # CONFIGURATION - using ENV

    # PLUGIN INIT - extensions for "app" object
    bs = Bootstrap5(app) # connects "Flask app" and "Bootstrap"
    
    # REGISTRATION (Blueprints / Routes)

    # APP CONTEXT - where we need to know the structure of the app 

    return app

2. **Linking with the base template (base.html):**
- Thanks to modifying the "base" template, from which other HTML files inherit, the changes will be immediately reflected in all descendants.

In [ ]:
<!-- base.html -->

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block page_title %} Title {% endblock page_title %}</title> <!-- místo pro dynamický obsah -->
    
    {{ bootstrap.load_css() }} <!-- Bootstrap CSS -->
</head>

<body>
    {% include 'header.html' %} <!-- zde vložíme obsah z header.html -->
    {% block page_content %} Content {% endblock page_content %} <!-- místo pro dynamický obsah -->
    {% include 'footer.html' %} <!-- zde vložíme obsah z footer.html -->

    {{ bootstrap.load_js() }} <!-- Bootstrap JS -->
</body>

</html>

Now we can use all **CSS classes** and **HTML elements** and components in our templates (for example, navigation, main page layout (heroes), footer and header sections, as well as elements like buttons ..)

- see documentation: https://getbootstrap.com/docs/5.3/getting-started/introduction/

### Bootstrap principle:
We can copy the **Bootstrap HTML code** of selected elements:
- from https://getbootstrap.com/docs/5.3/examples/

And paste it into the appropriate templates of our project (again, we will use inheritance = we will primarily edit the code in the base, header, and footer templates from which our other pages "inherit"). Further HTML modifications can, of course, be made on a specific page.
This way, we can quickly create a **professional look**.

## 9.4. Mission Start: Deploying the Application

The time has come to launch our application from the local simulator into the real orbit – the internet. For deployment, there are many "spaceports" (hosting platforms), each with its own advantages.

**Render (our choice)**: a modern and flexible platform with a clear interface and a good free tier for smaller projects - integration with GitHub is ideal for development.

**Heroku**: a long-standing standard in this category, works on a similar principle to Render.

**PythonAnywhere**: a good choice for beginners, plus it is focused purely on Python - it allows managing and editing the application directly in a web browser.

For our mission, we will demonstrate the process on the **Render.com** platform.

### 9.4.1 Step by step Deployment (Render):

1. **Final plans on GitHub:**
    * We make sure we have the final version of our project (including requirements.txt and Procfile files) uploaded to our GitHub repository.

2. **Creating an account on Render.com:**
    * We register and connect our GitHub account.

3. **Preparing the launchpad:**
   * In the main panel (Dashboard), we click on "New +" and select "Web Service".

4. **Connecting to the repository:**
   * We choose our GitHub repository with the selected Flask application.

5. **Pre-launch parameters:**
   * Render will try to set up most things on its own - but just in case, we check and fill in the following:
        * **Name:** unikátní jméno
        * **Runtime:** `Python 3`.
        * **Build Command:** `pip install -r requirements.txt`.
        * **Start Command:** `gunicorn wsgi:app`.
        * **Environment Variables:** in the "Advanced" section, we click on "Add Environment Variable" - here we enter our production values for `SECRET_KEY`, `DATABASE_URL`, and `APP_ENV`=production - **We never put sensitive data directly into the code on GitHub!**

6. **Launch!**
   * We click the "Create Web Service" button and watch the progress of the startup sequence in the "Logs" tab - in a few minutes, our project will be online at a unique URL address!

## Flask Course Summary
Congratulations on completing the basic training! You have successfully mastered all the key operations and are ready for independent missions.

### The Philosophy of Flask
* With Flask, we start with the absolute functional minimum.
* We add only what is needed and only when we want to add it.
* We retain full control over the project because we are not overwhelmed by the multitude of options and settings; project development is gradual.
* With great freedom comes great responsibility; we are responsible for the final architecture and structuring of the project ourselves.

### What You Take Away: Your New Skills
By completing this course, you have acquired a comprehensive set of skills for developing modern web applications. You can now:

* Create dynamic web pages using **Flask** and the **Jinja2** templating system.
* Process user input using robust **WTF-Forms**.
* Work with a database (save, read, update, and delete data) using **SQLAlchemy**.
* Structure code into a scalable architecture with an **Application Factory** and **Blueprints**.
* Design and build simple **REST APIs** for machine-to-machine communication.
* Prepare and **deploy an application** to a production environment.

Thanks to this, you are able to build projects such as dynamic **web applications/pages** (like blogs or portfolios), **data management applications** (user or product records), or a **backend system with a REST API** for mobile or JavaScript applications.


### Tips for future Improvement:

* **Increase System Robustness:** Now that your application is functional, focus on its quality and reliability. Explore writing automated tests (e.g., using `pytest`) to verify your systems' functionality after every change, and consider additional security features (authentication and authorization).

* **Publish Your Missions:** Your projects shouldn't remain secret. Create a portfolio on GitHub and upload your code. A successful application deployment to a service like Render is the best proof of your skills.

* **Become a Chronicler of Your Expeditions:** Document the development process, the problems you faced, and their solutions. Create and maintain `readme` files on GitHub as your logbook.

* **Update Your Technology:** Technology is constantly evolving. Return to your old code from time to time with new knowledge and perform refactoring. Every upgrade is the mark of an experienced engineer.

### **Project (Homework): Launch Sequence**

**Mission:** Our control system has successfully passed all simulations in the local environment. Before the final launch into orbit (deployment to a production server), we must perform the last critical adjustments: secure the configuration, improve the appearance, and prepare the startup sequence for the production engines. The goal is to transform our development prototype into a robust, secure, and professional-looking application ready for real-world operation.

`základní struktura naší aplikace`:
- my_project/
    - wsgi.py
    - app/
        - **Procfile**
        - **requirements.txt**
        - **.env**
        - _ _ init _ _.py
        - database.py
        - models/
            - _ _ init _ _.py
            - crewmember.py
        - data/
            - _ _ init _ _.py
            - asteroid.py
            - crew.py
        - templates/
        - static/
        - config/
        - main/ 
            - _ _ init _ _.py
            - routes.py 
            - templates/ 
        - crew/
            - _ _ init _ _.py
            - routes.py 
            - forms.py
            - templates/
        - api/ 
            - _ _ init _ _.py
            - routes.py
        - /instance
            - my_database.db


- The file with our **SQLite DB** is in the default **instance** folder next to our application, which is the **root** folder.


1.  **Secure Configuration:** Modify `config.py` and `app/__init__.py` to load sensitive data (`SECRET_KEY`, `DATABASE_URL`, `FLASK_ENV`) from environment variables. Create an `.env` file for local development.

2.  **Production Dependencies:** Create the `requirements.txt` and `Procfile` files needed for Gunicorn.

3.  **Appearance Improvement:** Integrate **Bootstrap** into the application, modify `base.html`, and try to improve selected elements (e.g., navigation, footer, or page layout) using Bootstrap.

4.  **(Bonus) Mission Start:** Deploy the application to a service like **Render** or another hosting provider.

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom